In [1]:
# Dependencies
import wikipedia
import pandas as pd
from random import sample 
import time
from math import log10,sqrt

# Searched for random nba and 'big' football teams
list_searches_nba = ['Los Angeles Lakers', 'Golden State Warriors', 'Toronto Raptors',
                'Milwaukee Bucks', 'LA Clippers', 'Boston Celtics', 'Denver Nuggets',
                'Utah Jazz', 'Miami Heat', 'Houston Rockets', 'Dallas Mavericks',
                    'Atlanta Hawks', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls',
                    'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Memphis Grizzlies',
                    'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder',
                    'San Antonio Spurs', 'Orlando Magic']
list_searches_cl = ['Manchester City', 'Paris Saint Germain', 'Real Madrid', 'FC Barcelona',
                    'Liverpool', 'Juventus', 'Valancia', 'Atalanta Bergamo',
                    'FC Bayern Munchen', 'Atlético Madrid', 'SSC Napoli', 'Borussia Dortmund',
                    'Olympique Lyon', 'Tottenham Hotspur', 'RasenBallsport Leipzig', 'Ajax Amsterdam',
                    'Dinamo Zagreb', 'APOEL Nicosia', 'Dynamo kiev', 'Bayer 04 Leverkusen', 'Internazionale',
                    'FC Red Bull Salzburg', 'Galatasaray', 'Olympiacos', 'Hoffenhheim', 'Schalke 04']

# Function for importaning the pages
def import_wikipedia(list_searches):
    data_searches, count = [], 0
    for search in list_searches:
        try:
            p = wikipedia.page(search)
            data_searches.append(p.content)
        except:
            count += 1
    return data_searches, count

data_nba, inv_nba = import_wikipedia(list_searches_nba)
data_cl, inv_cl = import_wikipedia(list_searches_cl)

# List of stopwords for removing stopwords
stopwords= ['a','able','about','across','after','all','almost','also','am','among','an','and','any','are','as','at','be','because','been','but','by',
            'can','cannot','could','dear','did','do','does','either','else','ever','every','for','from','get','got','had','has','have','he','her','hers',
            'him','his','how','however','i','if','in','into','is','it','its','just','least','let','like','likely','may','me','might','most','must','my',
            'neither','no','nor','not','of','off','often','on','only','or','other','our','own','rather','said','say','says','she','should','since','so',
            'some','than','that','the','their','them','then','there','these','they','this','tis','to','too','twas','us','wants','was','we','were','what',
            'when','where','which','while','who','whom','why','will','with','would','yet','you','your']

In [2]:
# This is for determining the classes based on filenames
class1 = list_searches_nba
class2 = list_searches_cl

In [23]:
# first reducer, counts the words, term n (global)
def reducer1(map_result):
    current_word = None
    current_count = 0
    word = None
    final_output = []

    for j in map_result:
        for line in j:
            line = line.strip()
            # because the list is sorted alphabetically, this works (this is also identical behavior in MapReduce for Hadoop)
            word, count = line.split('\t', 1)
            try:
                count = int(count)
            except ValueError:
                continue
                
            if current_word == word:
                current_count += count
            else:
                if current_word:
                    output = '%s \t %s' % (current_word, current_count)
                    final_output.append(output)
                current_count = count
                current_word = word
    # Appending the word counts
    if current_word == word:
        output = '%s \t %s' % (current_word, current_count)
        final_output.append(output)
    return final_output

In [24]:
# %%time
# map_result1 = mapper1(datasets, lst_search)
# reduce_result1 = reducer1(map_result1)

In [25]:
# second mapper arranges the documents for computhing N (total number of words in document)
def mapper2(data_reducer):
    final_output = []
    for line in data_reducer:
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        wordfilename,count=line.split('\t',1)
        wordfilename = wordfilename.strip()
        try:
            class_word,filename=wordfilename.split('*')
        except:
            pass
        try:
            clss,word=class_word.split('|')
        except:
            pass
        word = word.strip()
        z=word+' '+count+' | '+clss;
        output = '%s \t %s' % (filename, z)
        final_output.append(output)

    return final_output

In [26]:
# computes N (total words in a document) by summing the n's per document
def reducer2(map_result):
    current_word = None
    prev_filename = None
    current_count = 0
    word = None
    N=0
    df={}
    l1=[]
    final_output = []

    for line in map_result:
        line = line.strip()
        l1.append(line)
        filename,wordcount = line.split('\t', 1)
        wordcount_clss = wordcount.strip()
        wordcount,clss = wordcount.split('|', 1)
        word,count = wordcount.split()
        word = word.strip()
        clss = clss.strip()
        clss=int(clss)
        count = count.strip()
        count=int(count)
        if prev_filename == filename:
            N+=count
        else:
            if prev_filename != None:
                df[prev_filename]=N
            N=0
            prev_filename = filename
    df[prev_filename]=N

    for h in l1:
        filename,wordcount = h.split('\t', 1)
        wordcount_clss = wordcount.strip()
        wordcount,clss = wordcount.split('|', 1)
        word,count = wordcount.split()
        for k in df:
            if filename == k:
                wf=word+' '+filename
                nN=count+' '+str(df[k])+'|'+clss
                output = '%s \t %s' % (wf,nN)
                final_output.append(output)
    return final_output

In [27]:
# %%time
# map_result2 = mapper2(reduce_result1)
# reduce_result2 = reducer2(map_result2)

In [28]:
# Third mapper arranges words again, so class specific document frequencies can be computed
def mapper3(reduce_result):
    output_list = []
    for line in reduce_result:
        line = line.strip()
        wf,nN_clss=line.split('\t',1)
        w,f=wf.split(' ',1)
        nN_clss = nN_clss.strip()
        nN, clss = nN_clss.split('|')
        z=f+' * '+nN+' '+str(1)+' | '+clss
        output = '%s \t %s' % (w,z)
        output_list.append(output)
    final_output = sorted(output_list)
    return final_output

In [29]:
# Calculate class-specific document frequency
def reducer3(map_result):
    lst1 = []
    lst2 = []
    final_output = []
    prev_word = None
    count = 1 
    word = None
    for line in map_result:
        line = line.strip()
        w,clss = line.split('|')
        w=w.strip()
        clss=clss.strip()
        clss=int(clss)
        if clss==1:
            output = w+' | '+str(clss)
            lst1.append(output)
        else:
            output = w+' | '+str(clss)
            lst2.append(output)
    lst_total = [lst1+lst2][0]

    prev_word = None
    count = 1 
    word = None
    final_output = []
    for line in lst_total:
        line = line.strip()
        w,z= line.split('\t', 1)
        w = w.strip()
        z = z.strip()
        f,nNc_clss = z.split('*',1)
        f = f.strip()
        nNc_clss = nNc_clss.strip()
        nNc,clss=nNc_clss.split('|',1)
        nNc.strip()
        clss.strip()
        n,Nc=nNc.split(' ',1)
        N,c=Nc.split(' ',1)
        if prev_word == w:
            count += int(c)
        else:
            if prev_word != None:
                output=w+' '+f+' \t '+n+' '+N+' '+str(count)+' | '+str(clss)
                final_output.append(output)
            count=1
            prev_word = w
    return final_output

In [30]:
# %%time
# map_result3 = mapper3(reduce_result2)

In [31]:
# %%time
# reduce_result3 = reducer3(map_result3)

In [32]:
# Sorting the words for adding the class-specific document frequencies together
def mapper4(reduce_result):
    counts_clss1 = []
    wfs_clss1 = []
    counts_clss2 = []
    wfs_clss2 = []
    final_output = []
    for line in reduce_result:
        line = line.strip()
        wf_nNc, clss = line.split('|',1)
        wf,nNc = wf_nNc.split('\t',1)
        nNc = nNc.strip()
        n,N,fkr = nNc.split(' ')
        clss = clss.strip()
        clss = int(clss)
        wf = wf.strip()
        w,f = wf.split(' ',1)
        output = w+ ' '+str(clss)+' \t '+n+' '+N+' \\t ' +f+' \\\t '+fkr
        final_output.append(output)
    final_output = sorted(final_output)
    
    return final_output

In [33]:
# map_result4 = mapper4(reduce_result3)

In [34]:
# Adding the class-specific document frequencies together
def reducer4(map_result):
    final_output = []
    final_output2 = []
    prev_word = None
    prev_fkr = '1'
    for line in map_result:
        line = line.strip()
        w_class, nN_doc_fkr = line.split('\t',1)
        nN_doc, fkr = nN_doc_fkr.split('\\\t',1)
        fkr = fkr.strip()
        w_class = w_class.strip()
        w,clss = w_class.split(' ', 1)
        w = w.strip()
        if prev_word == w:
            total = str(fkr)+' '+str(prev_fkr)
        elif (w == prev_word and len(str(prev_fkr)) == 1):
            total = str(fkr)+' '+str(prev_fkr)
        else:
            total = str(fkr)
        prev_fkr = fkr
        prev_word = w
        output = w_class+' \t '+nN_doc+' \\\t '+str(total)
        final_output.append(output)

    prev_word = None
    prev_fkr = '1'
    for line in reversed(final_output):
        line = line.strip()
        w_class, nN_doc_fkr = line.split('\t',1)
        nN_doc, fkr = nN_doc_fkr.split('\\\t',1)
        fkr = fkr.strip()
        w_class = w_class.strip()
        w,clss = w_class.split(' ', 1)
        w = w.strip()
        if (prev_word == w and len(str(prev_fkr)) > 1):
            total = str(prev_fkr)
        else:
            total = str(fkr)
        prev_fkr = fkr
        prev_word = w
        output = w_class+' \t '+nN_doc+' \\\t '+str(total)
        final_output2.append(output)
    return final_output2

In [35]:
# reduce_result4 = reducer4(map_result4)

In [49]:
# Calculating the TF-IGM values per term
def mapper5(data_reducer):
    lmbd = 1.5
    final_output = []
    output_wclass = []
    output_igm = []
    for line in data_reducer:    
        line = line.strip()
        rest, fkrm = line.split('\\\t',1)
        fkrm = fkrm.strip()
        w_clss_nN,doc =  rest.split('\\t',1)
        w_clss_nN = w_clss_nN.strip()
        w_class, nN = w_clss_nN.split('\t',1)
        nN = nN.strip()
        n, N = nN.split(' ', 1)
        n = int(n)
        N = int(N)
        fkrm = fkrm.strip()
        if len(fkrm) > 2:
            if " " in fkrm:
                fkr1, fkr2 = fkrm.split(" ", 1)
                fkr1, fkr2 = int(fkr1), int(fkr2)
                if N == 0:
                    N = 1
                igm = n/N * (1+lmbd*(max([fkr1, fkr2])/sum([fkr1+fkr2])))
            else:
                igm = 0
        else:
            if N == 0:
                N = 1
            igm = (n/N)*(1+lmbd*1)
        output = w_class+ ' '+doc+' \t '+str(igm)
#         output_wclass.append(w_class)
#         output_igm.append(igm)
        final_output.append(output)
        
    return final_output

In [37]:
def parallel_tfigm(data): #, searches):
    map_result1 = mapper1(data) #, searches)
    reduce_result1 = reducer1(map_result1)
    map_result2 = mapper2(reduce_result1)
    reduce_result2 = reducer2(map_result2)
    map_result3 = mapper3(reduce_result2)
    reduce_result3 = reducer3(map_result3)
    map_result4 = mapper4(reduce_result3)
    reduce_result4 = reducer4(map_result4)
    final = mapper5(reduce_result4)
    
    return final

In [38]:
datasets = data_nba*100 + data_cl*100

# Merging the lists
lst_search = list_searches_nba + list_searches_cl

In [39]:
# from functools import partial
from multiprocessing import Pool
import time
import os

In [40]:
# First mapper function, attaching class, removing stopwords, and sorting the words (alphabetically)
def mapper1(data):
    final_output = []
    for i in range(len(data)):
        output_not_sorted = []
        filename = sample(lst_search, 1)[0]
#         filename = lst_search[i]
        if filename in class1:
            given_class = 1
        else:
            given_class = 2
#         given_class = sample([1, 2], 1)
        
        # Remove white space
        line = data[i].strip()
        # split into words
        words = line.split()
        for word in words:
            word=word.lower()
            if word not in stopwords:
                # attaching the class
                z=str(given_class)+' | '+word+' * '+filename
                output = '%s \t %s' % (z, 1)
                output_not_sorted.append(output)
        sorted_output = sorted(output_not_sorted)
        final_output.append(sorted_output)
    return final_output

In [41]:
print("oke")

oke


In [50]:
with Pool(processes=6) as pool:
    running_times = []
    for i in range(100, 1100, 100):
        new_data = datasets[:i]# sample(datasets,i)
#         lst_searches = lst_search[:i] # sample(lst_search, i)
        start_time = time.time()
#         func = partial(parallel_tfigm, new_data)
        pool.map(parallel_tfigm, new_data)
        print(time.time() - start_time)
        running_times.append(time.time() - start_time)

10.329134225845337


Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-21:
Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run(

KeyboardInterrupt: 

In [52]:
len(new_data)

700

In [ ]:
# Local TF-IGM
57.18224501609802
112.47146010398865
165.73978686332703
221.77069091796875
270.4036648273468
320.8091850280762
400.92387413978577
451.65632700920105
511.9483811855316
555.0262262821198

In [ ]:
# Parallel TF-IGM
10.912653923034668
25.431442975997925
36.806010246276855
47.119266748428345
56.70264911651611
70.51972198486328
81.978276014328
91.14404082298279
120.51627683639526
121.98761200904846

In [53]:
with Pool(processes=1) as pool:
    running_times = []
    for i in range(700, 1100, 100):
        new_data = datasets[:i]# sample(datasets,i)
#         lst_searches = lst_search[:i] # sample(lst_search, i)
        start_time = time.time()
#         func = partial(parallel_tfigm, new_data)
        pool.map(parallel_tfigm, new_data)
        print(time.time() - start_time)
        running_times.append(time.time() - start_time)

400.92387413978577
451.65632700920105
511.9483811855316
555.0262262821198


In [55]:
with Pool(processes=6) as pool:
    running_times = []
    for i in range(100, 1100, 100):
        new_data = datasets[:i]# sample(datasets,i)
#         lst_searches = lst_search[:i] # sample(lst_search, i)
        start_time = time.time()
#         func = partial(parallel_tfigm, new_data)
        pool.map(parallel_tfigm, new_data)
        print(time.time() - start_time)
        running_times.append(time.time() - start_time)

10.912653923034668
25.431442975997925
36.806010246276855
47.119266748428345
56.70264911651611
70.51972198486328
81.978276014328
91.14404082298279
120.51627683639526
121.98761200904846


In [56]:
new_data = datasets[:900]

In [57]:
len(new_data)

900

In [ ]:
# 2 cores: 245.01554584503174
# 3 cores: 169.88558411598206
# 4 cores: 137.16499018669128
# 5 cores: 123.00809288024902

In [59]:
for i in range(4, 6, 1):
    with Pool(processes=i) as pool:
        start_time = time.time()
        result = pool.map(parallel_tfigm, new_data)
        print(time.time() - start_time)

137.16499018669128
123.00809288024902


In [123]:
with Pool(processes=1) as pool:
    running_times = []
    for i in range(100, 1100, 100):
        new_data = datasets[:i]# sample(datasets,i)
#         lst_searches = lst_search[:i] # sample(lst_search, i)
        start_time = time.time()
#         func = partial(parallel_tfigm, new_data)
        pool.map(parallel_tfigm, new_data)
        print(time.time() - start_time)
        running_times.append(time.time() - start_time)

138.08856916427612
278.4285681247711
416.02373123168945
555.573157787323
720.0035171508789
870.6720798015594
991.5431771278381
1151.6833758354187
1274.6585099697113
1409.7844321727753


In [ ]:
# parallel (6 cores)
30.135493993759155
54.48365592956543
80.22214913368225
106.95011878013611
136.95192098617554
168.36995792388916
195.76798677444458
221.30671095848083
253.11553406715393
289.38061785697937

In [ ]:
# local 1 core
138.08856916427612
278.4285681247711
416.02373123168945
555.573157787323
720.0035171508789
870.6720798015594
991.5431771278381
1151.6833758354187
1274.6585099697113
1409.7844321727753

In [ ]:
2.1351287364959717
2.1325666904449463
2.2169289588928223
2.276876926422119
2.427320957183838
2.4833920001983643
2.4856011867523193
2.629715919494629

In [ ]:
1.1243817806243896
1.223736047744751
1.2502851486206055
1.3273632526397705
1.3809452056884766
1.4344279766082764
1.507964849472046
1.619715690612793

In [27]:
# # %%time
# running_times = []
# for i in range(5, 45, 5): # 51
#     new_data = sample(datasets,i)
#     lst_searches = sample(lst_search, i)
#     start_time = time.time()
#     map_result1 = mapper1(new_data, lst_searches)
#     reduce_result1 = reducer1(map_result1)
#     map_result2 = mapper2(reduce_result1)
#     reduce_result2 = reducer2(map_result2)
#     map_result3 = mapper3(reduce_result2)
#     reduce_result3 = reducer3(map_result3)
#     map_result4 = mapper4(reduce_result3)
#     reduce_result4 = reducer4(map_result4)
#     final = mapper5(reduce_result4)
#     running_times.append(time.time() - start_time)

In [28]:
running_times

[0.2533600330352783,
 0.45272302627563477,
 0.5005459785461426,
 0.7690169811248779,
 0.9512901306152344,
 1.1218140125274658,
 1.3915767669677734,
 1.5869920253753662]

In [29]:
len(datasets)

45